In [1]:
!which python

/Users/priya/Downloads/UC Davis/Assignments/venv/bin/python


In [6]:
pip install -r requirements.txt

You should consider upgrading via the '/Users/priya/Downloads/UC Davis/Assignments/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Input the dish item from the user

Open the search result from Doordash website

In [ ]:
import undetected_chromedriver as uc
import chromedriver_py  # this imports the chromedriver path
import time
import random
from selenium.webdriver.chrome.options import Options

# Set up Chrome options
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.6943.142 Safari/537.36")

# Get ChromeDriver path from chromedriver_py
driver_path = chromedriver_py.binary_path

# Set up undetected ChromeDriver with the ChromeDriver path
driver = uc.Chrome(driver_executable_path=driver_path, options=options)

# Open the DoorDash search page with the search query
search_input = input("Enter the store type or keyword to search on DoorDash (e.g., 'sushi'): ")
url = f"https://www.doordash.com/search/store/{search_input}?event_type=search"
driver.get(url)

# Add some delay before interacting with the page
time.sleep(random.uniform(2, 4))

# Execute JavaScript to hide webdriver detection
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Give time for the page to load
time.sleep(random.uniform(2, 4))

# Close browser
input("Press Enter to close the browser...")
driver.quit()


This page has a lot of restaurants that serves sushi

- Fetch the restaurant id from the element

In [10]:
import undetected_chromedriver as uc  # Correct import statement
import chromedriver_py  # This imports the chromedriver path
import time
import random
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Set up Chrome options
options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# Get the ChromeDriver path using chromedriver_py
driver_path = chromedriver_py.binary_path

# Initialize undetected ChromeDriver with the ChromeDriver path
driver = uc.Chrome(executable_path=driver_path, options=options, timeout=120)

# Open the DoorDash search page with the search query
search_input = input("Enter the store type or keyword to search on DoorDash (e.g., 'sushi'): ")
url = f"https://www.doordash.com/search/store/{search_input}?event_type=search"
driver.get(url)

# Add some delay before interacting with the page to mimic human-like browsing
time.sleep(random.uniform(2, 4))  # Wait between 3 to 7 seconds randomly

# Execute JavaScript to hide webdriver detection
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Give time for the page to load and handle any security checks that might still appear
time.sleep(random.uniform(2, 4))  # Wait again to mimic a real user

# Fetch all restaurant ID links
try:
    # Use XPath to find all <a> tags containing the store IDs
    restaurant_elements = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/store/')]")
    
    # Extract the restaurant IDs from the href attributes
    restaurant_ids = []
    for element in restaurant_elements:
        href = element.get_attribute('href')
        # Extract the ID from the href by splitting the string
        if '/store/' in href:
            restaurant_id = href.split('/store/')[1].split('?')[0]
            restaurant_ids.append(restaurant_id)

    # Remove duplicates by converting to a set and then back to a list
    unique_restaurant_ids = list(set(restaurant_ids))

    # Print the first 5 unique restaurant IDs
    print(f"First 5 restaurant IDs for '{search_input}':")
    for restaurant_id in unique_restaurant_ids[:5]:
        print(restaurant_id)

    # Print total number of unique restaurant IDs found
    print(f"\nTotal number of unique restaurant IDs found: {len(unique_restaurant_ids)}")

except Exception as e:
    print(f"Error extracting restaurant IDs: {e}")

# Wait for the user to press Enter before closing the browser
input("\nPress Enter to close the browser...")

# Close the browser after the user presses Enter
driver.quit()


First 5 restaurant IDs for 'sushi':
193470
240183
956708
718398
869275

Total number of unique restaurant IDs found: 10


Create a JSON for every restaurant and append all the details - ID, Name, Ratings, No of Ratings, Delivery Time, Distance

In [30]:
import undetected_chromedriver as uc  # Correct import statement
import chromedriver_py  # This imports the chromedriver path
import time
import random
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Chrome options
options = Options()

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# Get the ChromeDriver path using chromedriver_py
driver_path = chromedriver_py.binary_path

# Initialize undetected ChromeDriver with the ChromeDriver path
driver = uc.Chrome(executable_path=driver_path, options=options, timeout=120)

# Open the DoorDash search page with the search query
search_input = input("Enter the store type or keyword to search on DoorDash (e.g., 'sushi'): ")
url = f"https://www.doordash.com/search/store/{search_input}?event_type=search"
driver.get(url)

# Add some delay before interacting with the page to mimic human-like browsing
time.sleep(random.uniform(2, 4))  # Wait between 2 to 4 seconds randomly

# Execute JavaScript to hide webdriver detection
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Wait for page to load
time.sleep(random.uniform(2, 4))

# Initialize the list to hold restaurant data
restaurants_data = []

# Fetch all restaurant links
try:
    # Find all restaurant anchor tags
    restaurant_links = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//a[starts-with(@href, '/store/') or starts-with(@href, '/convenience/store/')]"))
    )
    
    # Process each restaurant
    for link in restaurant_links:
        try:
            # Extract restaurant ID from href
            href = link.get_attribute('href')
            restaurant_id = href.split('/store/')[-1].split('?')[0]
            
            # Extract restaurant name
            try:
                restaurant_name = link.find_element(By.XPATH, ".//span[@data-telemetry-id='store.name']").text.strip()
            except:
                restaurant_name = ''
            
            # Extract ratings
            try:
                rating = link.find_element(By.XPATH, ".//span[contains(@class, 'sc-cfbb81c7-1') and contains(@class, 'sc-cfbb81c7-2')]").text.strip()
            except:
                rating = ''
            
            # Extract number of ratings
            try:
                no_of_ratings = link.find_element(By.XPATH, ".//span[contains(@class, 'sc-cfbb81c7-1')]").text.strip('()')
            except:
                no_of_ratings = ''
            
            # Extract distance
            try:
                distance = link.find_element(By.XPATH, ".//span[contains(@class, 'sc-ab1e2fe8-18') and contains(text(), 'mi')]").text.strip()
            except:
                distance = ''
            
            # Extract delivery time
            try:
                delivery_time = link.find_element(By.XPATH, ".//span[contains(@class, 'sc-ab1e2fe8-18') and contains(text(), 'min')]").text.strip()
            except:
                delivery_time = ''
            
            # Create restaurant data dictionary
            restaurant_data = {
                "restaurant_id": restaurant_id,
                "restaurant_name": restaurant_name,
                "ratings": rating,
                "no_of_ratings": no_of_ratings,
                "distance": distance,
                "delivery_time": delivery_time,
                "URL": href
            }
            
            # Append to restaurants list
            restaurants_data.append(restaurant_data)
        
        except Exception as link_error:
            print(f"Error processing restaurant link: {link_error}")
    
    # Print results
    print(f"Restaurants found for '{search_input}':")
    for restaurant in restaurants_data:
        print(restaurant)
    
    print(f"\nTotal number of restaurants found: {len(restaurants_data)}")

except Exception as e:
    print(f"Error extracting restaurant data: {e}")

# Wait for the user to press Enter before closing the browser
input("\nPress Enter to close the browser...")

# Close the browser after the user presses Enter
driver.quit()

Restaurants found for 'sushi':
{'restaurant_id': '240255', 'restaurant_name': '', 'ratings': '', 'no_of_ratings': '', 'distance': '', 'delivery_time': '', 'URL': 'https://www.doordash.com/store/240255?cursor=eyJzZWFyY2hfaXRlbV9jYXJvdXNlbF9jdXJzb3IiOnsicXVlcnkiOiJzdXNoaSIsIml0ZW1faWRzIjpbXSwic2VhcmNoX3Rlcm0iOiJzdXNoaSIsInZlcnRpY2FsX2lkIjotOTk5LCJ2ZXJ0aWNhbF9uYW1lIjoiYWxsIn0sInN0b3JlX3ByaW1hcnlfdmVydGljYWxfaWRzIjpbMSw0LDE3N119&pickup=false'}
{'restaurant_id': '240255', 'restaurant_name': 'Teabo Cafe', 'ratings': '4.5', 'no_of_ratings': '4.5', 'distance': '1.3 mi', 'delivery_time': '26 min', 'URL': 'https://www.doordash.com/store/240255?cursor=eyJzZWFyY2hfaXRlbV9jYXJvdXNlbF9jdXJzb3IiOnsicXVlcnkiOiJzdXNoaSIsIml0ZW1faWRzIjpbXSwic2VhcmNoX3Rlcm0iOiJzdXNoaSIsInZlcnRpY2FsX2lkIjotOTk5LCJ2ZXJ0aWNhbF9uYW1lIjoiYWxsIn0sInN0b3JlX3ByaW1hcnlfdmVydGljYWxfaWRzIjpbMSw0LDE3N119&pickup=false'}
{'restaurant_id': '956708', 'restaurant_name': '', 'ratings': '', 'no_of_ratings': '', 'distance': '', 'delivery_ti

Navigate into each restaurant through the url created from restaurant ID

In [19]:
import undetected_chromedriver as uc  # Correct import statement
import time
import random
from selenium.webdriver.chrome.options import Options

# Set up Chrome options (without headless mode)
options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# Initialize undetected ChromeDriver
driver = uc.Chrome(options=options)

# Open the DoorDash store page for a specific restaurant ID
restaurant_id = '956708'
restaurant_url = f"https://www.doordash.com/store/{restaurant_id}"
driver.get(restaurant_url)

# Wait for the page to load (optional, you can adjust the wait time based on your needs)
time.sleep(random.uniform(3, 7))  # Wait between 3 to 7 seconds to mimic human-like browsing

# Wait for the URL to update automatically to the new format (store name + restaurant ID)
time.sleep(3)

# Extract the updated URL from the browser
updated_url = driver.current_url

# Extract the restaurant name from the URL
# The format of the URL is: https://www.doordash.com/store/<restaurant_name>/<restaurant_id>/
# Split the URL to get the restaurant name and ID
url_parts = updated_url.split('/')

# The restaurant name is the part before the restaurant ID in the URL
restaurant_name = url_parts[4]  # The name will be in the 4th part of the URL

# Print the restaurant name and ID
print(f"Restaurant Name: {restaurant_name}")
print(f"Restaurant ID: {restaurant_id}")

# Wait for the user to press Enter before closing the browser
input("\nPress Enter to close the browser...")

# Close the browser after the user presses Enter
driver.quit()


Restaurant Name: i-love-sushi-davis-956708
Restaurant ID: 956708


Search the item from the restaurant page

In [34]:
import undetected_chromedriver as uc
import chromedriver_py
import time
import random
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Sample restaurant data (you can replace this with your actual JSON data)
# sample_restaurant_data = [
#     {
#         "restaurant_id": "240255",
#         "restaurant_name": "Teabo Cafe",
#         "ratings": "4.5",
#         "no_of_ratings": "6k+",
#         "distance": "1.8 mi",
#         "delivery_time": "23 min",
#         "URL": "https://www.doordash.com/store/240255?cursor=eyJzZWFyY2hfaXRlbV9jYXJvdXNlbF9jdXJzb3IiOnsicXVlcnkiOiJzdXNoaSIsIml0ZW1faWRzIjpbXSwic2VhcmNoX3Rlcm0iOiJzdXNoaSIsInZlcnRpY2FsX2lkIjotOTk5LCJ2ZXJ0aWNhbF9uYW1lIjoiYWxsIn0sInN0b3JlX3ByaW1hcnlfdmVydGljYWxfaWRzIjpbMSw0LDE3N119&pickup=false"
#     }
# ]

# Set up Chrome options
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# Get the ChromeDriver path using chromedriver_py
driver_path = chromedriver_py.binary_path

# Initialize undetected ChromeDriver
driver = uc.Chrome(executable_path=driver_path, options=options)

try:
    # Take the first restaurant from the list (for simplicity)
    # restaurant = sample_restaurant_data[0]

    restaurant_url = 'https://www.doordash.com/store/240255?cursor=eyJzZWFyY2hfaXRlbV9jYXJvdXNlbF9jdXJzb3IiOnsicXVlcnkiOiJzdXNoaSIsIml0ZW1faWRzIjpbXSwic2VhcmNoX3Rlcm0iOiJzdXNoaSIsInZlcnRpY2FsX2lkIjotOTk5LCJ2ZXJ0aWNhbF9uYW1lIjoiYWxsIn0sInN0b3JlX3ByaW1hcnlfdmVydGljYWxfaWRzIjpbMSw0LDE3N119&pickup=false'
    driver.get(restaurant_url)
    
    time.sleep(random.uniform(2, 4))  # Wait between 3 to 7 seconds to mimic human-like browsing
    
    # print(f"Restaurant Name: {restaurant['restaurant_name']}")
    # print(f"Restaurant ID: {restaurant['restaurant_id']}")
    
    # Find search input using XPath
    search_bar = driver.find_element(By.XPATH, "//input[@id='item-search-field']")
    
    # Input search term
    search_input = input("Enter the item you want to search for: ")
    
    # Clear any existing text in the search bar
    search_bar.clear()
    
    # Enter the search term into the search bar
    search_bar.send_keys(search_input)
    
    # Press Enter to trigger the search
    search_bar.send_keys(Keys.RETURN)
    
    # Wait for search results to load
    time.sleep(random.uniform(2, 4))
    
    print(f"Search for '{search_input}' completed.")

except Exception as e:
    print(f"An error occurred: {e}")
    
finally:
    # Wait for the user to press Enter before closing the browser
    input("\nPress Enter to close the browser...")
    
    # Close the browser
    driver.quit()

An error occurred: HTTPConnectionPool(host='localhost', port=51855): Read timed out. (read timeout=120)


In [35]:
import undetected_chromedriver as uc
import chromedriver_py
import time
import random
import traceback
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

# Set up Chrome options
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# Get the ChromeDriver path using chromedriver_py
driver_path = chromedriver_py.binary_path

def navigate_and_search():
    # Initialize undetected ChromeDriver
    driver = None
    try:
        # Set up Chrome options to prevent timeout
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        
        # Initialize the driver with extended timeout
        driver = uc.Chrome(
            executable_path=driver_path, 
            options=options, 
            timeout=180  # Increased timeout to 3 minutes
        )
        
        # Set page load timeout
        driver.set_page_load_timeout(60)
        
        # Navigate to the restaurant URL
        restaurant_url = 'https://www.doordash.com/store/240255?cursor=eyJzZWFyY2hfaXRlbV9jYXJvdXNlbF9jdXJzb3IiOnsicXVlcnkiOiJzdXNoaSIsIml0ZW1faWRzIjpbXSwic2VhcmNoX3Rlcm0iOiJzdXNoaSIsInZlcnRpY2FsX2lkIjotOTk5LCJ2ZXJ0aWNhbF9uYW1lIjoiYWxsIn0sInN0b3JlX3ByaW1hcnlfdmVydGljYWxfaWRzIjpbMSw0LDE3N119&pickup=false'
        driver.get(restaurant_url)
        
        # Wait for page to load completely
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )
        
        # Add random delay to mimic human behavior
        time.sleep(random.uniform(2, 4))
        
        # Find search input using multiple methods
        try:
            # Try WebDriverWait first
            search_bar = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "//input[@id='item-search-field']"))
            )
        except TimeoutException:
            # Fallback to direct find_element
            search_bar = driver.find_element(By.XPATH, "//input[@id='item-search-field']")
        
        # Input search term
        search_input = input("Enter the item you want to search for: ")
        
        # Clear any existing text in the search bar
        search_bar.clear()
        
        # Enter the search term into the search bar
        search_bar.send_keys(search_input)
        
        # Press Enter to trigger the search
        search_bar.send_keys(Keys.RETURN)
        
        # Wait for search results to load
        time.sleep(random.uniform(2, 4))
        
        print(f"Search for '{search_input}' completed.")
        
        # Optional: Add a screenshot for verification
        driver.save_screenshot('search_results.png')
        
        return True
    
    except Exception as e:
        print("Detailed Error Information:")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Message: {str(e)}")
        print("\nFull Traceback:")
        traceback.print_exc()
        
        # Optional: Take a screenshot of the error state
        if driver:
            try:
                driver.save_screenshot('error_screenshot.png')
            except:
                pass
        
        return False
    
    finally:
        # Ensure driver is closed
        if driver:
            driver.quit()

# Main execution
if __name__ == "__main__":
    try:
        success = navigate_and_search()
        
        if not success:
            print("Navigation and search failed.")
        
    except Exception as e:
        print(f"Unexpected error: {e}")

Search for 'sushi' completed.
